# Necessary imports

In [58]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Reading the file using pandas

In [59]:
filename = 'IMDB Dataset.csv'
df = pd.read_csv(filename, index_col=False)

# Dimensional analysis of the data.

In [60]:
df.shape

(50000, 2)

# Analysis of the sentiment column resulting in 50-50 split.

In [61]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

# Reading the text completely.

In [62]:
print(f"Review : \n\n{df.review.iloc[0]} \n ")
print(f"Label  : {df.sentiment.iloc[0]}")

Review : 

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due

# Preparing data for splitting and dropping index column in test section.

In [63]:
df['sentiment'] = df['sentiment'].map({'positive':0,'negative':1})

train_df = df.sample(frac=0.8,random_state=100)
test_df = df.drop(train_df.index)

print(f"Train data shape: {train_df.shape}")
print(f"Test  data shape: {test_df.shape}")

Train data shape: (40000, 2)
Test  data shape: (10000, 2)


# Checking the difference between the train and test data.

In [64]:
print(test_df)

                                                  review  sentiment
9      If you like original gut wrenching laughter yo...          0
27     This film tried to be too many things all at o...          1
28     This movie was so frustrating. Everything seem...          1
40     It had all the clichés of movies of this type ...          1
44     This movie struck home for me. Being 29, I rem...          0
...                                                  ...        ...
49956  Lord, this sucked. There's a particular sort o...          1
49958  As someone who loves baseball history, especia...          1
49980  A stunning film of high quality.<br /><br />Ap...          0
49988  When I first tuned in on this morning news, I ...          1
49994  This is your typical junk comedy.<br /><br />T...          1

[10000 rows x 2 columns]


# Word Tokenizing and counting the word index, ultimately leading to the shape.

In [65]:
tokenizer  = tf.keras.preprocessing.text.Tokenizer(num_words=8000)
tokenizer.fit_on_texts(np.append(train_df.review.values,test_df.review.values))

word_index = tokenizer.word_index
nb_words = len(word_index) + 1

train_seq = tokenizer.texts_to_sequences(train_df.review)
test_seq = tokenizer.texts_to_sequences(test_df.review)

train_data = tf.keras.preprocessing.sequence.pad_sequences(train_seq, maxlen=100)
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_seq, maxlen=100)

print(f"Train data shape: {train_data.shape}")
print(f"Test  data shape: {test_data.shape}")

Train data shape: (40000, 100)
Test  data shape: (10000, 100)


In [66]:
len(word_index)

124252

In [67]:
train_label = train_df.sentiment.values
test_label = test_df.sentiment.values

# Defining the model

In [68]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(nb_words, 128),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(1)])

    model.compile(optimizer='adam',
                  loss=tf.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

model = create_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         15904384  
_________________________________________________________________
global_average_pooling1d_3 ( (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 15,912,705
Trainable params: 15,912,705
Non-trainable params: 0
_________________________________________________________________


# Defing the callback and the fitting the model on training data.

In [69]:
call_back = [tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",patience=2,
                                              verbose=1,restore_best_weights=True)]

model.fit(train_data, train_label, epochs=5, batch_size=128,
          validation_data = (test_data,test_label),
          callbacks=call_back)

Epoch 1/5
313/313 [==============================] - 88s 281ms/step - loss: 0.4404 - accuracy: 0.7436 - val_loss: 0.3177 - val_accuracy: 0.8519
Epoch 2/5
313/313 [==============================] - 91s 291ms/step - loss: 0.2757 - accuracy: 0.8798 - val_loss: 0.3187 - val_accuracy: 0.8522
Epoch 3/5
313/313 [==============================] - 95s 302ms/step - loss: 0.2467 - accuracy: 0.8947 - val_loss: 0.3281 - val_accuracy: 0.8547
Epoch 4/5
313/313 [==============================] - 122s 390ms/step - loss: 0.2304 - accuracy: 0.9029 - val_loss: 0.3526 - val_accuracy: 0.8545
Epoch 5/5
313/313 [==============================] - 86s 273ms/step - loss: 0.2204 - accuracy: 0.9080 - val_loss: 0.3734 - val_accuracy: 0.8503
Epoch 00005: early stopping


# Evaluating the model on the test data.

In [70]:
loss, accuracy = model.evaluate(test_data,test_label)

print(f"Accuracy : {accuracy}")
print(f"Loss     : {loss}")

313/313 [==============================] - 1s 4ms/step - loss: 0.3281 - accuracy: 0.8547
Accuracy : 0.8547000288963318
Loss     : 0.32811224460601807
